In [ ]:
pip install openai langchain langsmith langchain_community langchain-openai

In [ ]:
# 환경 변수 설정

import os

# OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

# LangSmith API Key & 프로젝트 이름
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "OCR-AI"  # 원하는 이름으로 지정

# 선택적으로 디버깅용 로깅 활성화
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
import base64

# 이미지 파일을 base64로 인코딩
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [13]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_core.tracers import ConsoleCallbackHandler

# 필드 목록 정의
fields = [
    "접수번호", "성명", "소속", "지위", "생년월일", "전화번호", "주소", "입사일자", "실 근속년 수",
    "신청금액", "자금용도", "대출기일", "상환예정일", "담보 여건", "신청날짜", "신청인"
]

# 프롬프트 템플릿
prompt_template = """
아래 이미지를 보고 다음 필드들을 추출해 JSON 형태로 작성해 주세요. 모든 값은 문자열로 반환하세요.
필드 목록:
{field_list}

반드시 아래 형식을 따르세요:

{{
  "접수번호": "...",
  "성명": "...",
  ...
  "신청인": "..."
}}
"""

prompt = prompt_template.format(field_list="\n".join(fields))

# 이미지 인코딩
image_path = "../documents/직원대출신청서_5개모음집_pages-to-jpg-0001.jpg"  # 로컬 이미지 경로
base64_image = encode_image(image_path)

# LangSmith 추적 핸들러
handler = ConsoleCallbackHandler()

# GPT-4o로 설정
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    callbacks=[handler],  # LangSmith로 자동 전송
)

# 메시지 생성 (텍스트 + 이미지)
message = HumanMessage(content=[
    {"type": "text", "text": prompt},
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
        }
    }
])

In [ ]:
# GPT 호출
response = llm.invoke([message])

In [16]:
print(response.content)

```json
{
  "접수번호": "20250401",
  "성명": "김하늘",
  "소속": "마케팅팀",
  "지위": "대리",
  "생년월일": "1990-06-15",
  "전화번호": "010-1234-5678",
  "주소": "서울특별시 강남구 테헤란로 123",
  "입사일자": "2018년 3월 1일",
  "실 근속년 수": "6년 1개월",
  "신청금액": "삼백만원 (₩ 3,000,000)",
  "자금용도": "의료비",
  "대출기일": "2025년 6월 1일",
  "상환예정일": "2026년 6월 1일",
  "담보 여건": "무담보",
  "신청날짜": "2025년 4월 22일",
  "신청인": "김하늘"
}
```
